In [6]:
# Added cell to set Working Directory to your location
import os
import re
import glob
import numpy as np
import pandas as pd
import json
import time

In [3]:
Complete_data = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/combined_csv.csv")

In [ ]:
Complete_data_sorted = Complete_data.sort_values(by=["SubjectID","Session","SessionSubsection"])

In [7]:
One_participant = pd.read_csv("/Volumes/SSD/00_Data_Processing/Pre_processed/00_Individuals_Flat/1754.csv")

In [18]:
One_participant.head(100)

,Unnamed: 0,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,timeStampGetVerboseData,eyeOpennessLeft,eyeOpennessRight,pupilDiameterMillimetersLeft,...,bodyTrackerPosition.z,bodyTrackerRotation.x,bodyTrackerRotation.y,bodyTrackerRotation.z,0,hitColliderType,hitObjectColliderBoundsCenter,hitObjectColliderName,hitPointOnObject,ordinalOfHit
0,133291,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.736757,1.000000,4.519638,...,-0.093050,8.100518,355.064697,0.794874,NaN,UnityEngine.MeshCollider,"{'x': -165.37255859375, 'y': -0.46650505065917...",terrain_O.001,"{'x': -62.10708236694336, 'y': 1.1084452867507...",1.0
1,133292,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.736757,1.000000,4.519638,...,-0.093050,8.100518,355.064697,0.794874,NaN,UnityEngine.MeshCollider,"{'x': -165.37255859375, 'y': -0.46650505065917...",terrain_O.001,"{'x': -62.10708236694336, 'y': 1.1084452867507...",1.0
2,133293,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.599419,1.000000,4.519638,...,-0.093022,8.135050,355.152496,0.800912,NaN,UnityEngine.MeshCollider,"{'x': -165.37255859375, 'y': -0.46650505065917...",terrain_O.001,"{'x': -62.059593200683594, 'y': 1.093759059906...",1.0
3,133294,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.558560,1.000000,4.509415,...,-0.093034,8.127725,355.173431,0.789557,NaN,UnityEngine.MeshCollider,"{'x': -165.37255859375, 'y': -0.46650505065917...",terrain_O.001,"{'x': -61.975582122802734, 'y': 1.049011945724...",1.0
4,133295,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.558560,1.000000,4.509415,...,-0.093121,8.043446,355.143066,0.795973,NaN,UnityEngine.MeshCollider,"{'x': -165.37255859375, 'y': -0.46650505065917...",terrain_O.001,"{'x': -61.9382438659668, 'y': 1.04141104221344...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,133386,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.708111,0.976150,3.618317,...,-0.096278,8.027719,355.109436,0.570223,NaN,UnityEngine.MeshCollider,"{'x': -68.42803955078125, 'y': 4.5190935134887...",Building_160,"{'x': -76.13035583496094, 'y': 3.5341017246246...",1.0
96,133387,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.708111,0.976150,3.618317,...,-0.096278,8.027719,355.109436,0.570223,NaN,UnityEngine.MeshCollider,"{'x': -61.46739196777344, 'y': 6.5191745758056...",Building_176,"{'x': -80.45380401611328, 'y': 3.8150935173034...",2.0
97,133388,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.708111,0.976150,3.618317,...,-0.096278,8.027719,355.109436,0.570223,NaN,UnityEngine.MeshCollider,"{'x': -68.42803955078125, 'y': 4.5190935134887...",Building_160,"{'x': -76.13035583496094, 'y': 3.5341017246246...",1.0
98,133389,1754,1,1,1.646740e+09,1.646740e+09,1.646740e+09,0.708111,0.976150,3.618317,...,-0.096278,8.027719,355.109436,0.570223,NaN,UnityEngine.MeshCollider,"{'x': -61.46739196777344, 'y': 6.5191745758056...",Building_176,"{'x': -80.45380401611328, 'y': 3.8150935173034...",2.0


In [15]:
print(One_participant.columns.tolist())

['Unnamed: 0', 'SubjectID', 'Session', 'SessionSubsection', 'timeStampDataPointStart', 'timeStampDataPointEnd', 'timeStampGetVerboseData', 'eyeOpennessLeft', 'eyeOpennessRight', 'pupilDiameterMillimetersLeft', 'pupilDiameterMillimetersRight', 'leftGazeValidityBitmask', 'rightGazeValidityBitmask', 'combinedGazeValidityBitmask', 'rayCastHitsCombinedEyes', 'rayCastHitsLeftEye', 'rayCastHitsRightEye', 'eyePositionCombinedWorld.x', 'eyePositionCombinedWorld.y', 'eyePositionCombinedWorld.z', 'eyeDirectionCombinedWorld.x', 'eyeDirectionCombinedWorld.y', 'eyeDirectionCombinedWorld.z', 'eyeDirectionCombinedLocal.x', 'eyeDirectionCombinedLocal.y', 'eyeDirectionCombinedLocal.z', 'eyePositionLeftWorld.x', 'eyePositionLeftWorld.y', 'eyePositionLeftWorld.z', 'eyeDirectionLeftWorld.x', 'eyeDirectionLeftWorld.y', 'eyeDirectionLeftWorld.z', 'eyeDirectionLeftLocal.x', 'eyeDirectionLeftLocal.y', 'eyeDirectionLeftLocal.z', 'eyePositionRightWorld.x', 'eyePositionRightWorld.y', 'eyePositionRightWorld.z', 'e

In [ ]:
Keep = ['timeStampDataPointStart', 'timeStampDataPointEnd', 'timeStampGetVerboseData','combinedGazeValidityBitmask','rayCastHitsCombinedEyes','eyePositionCombinedWorld.x', 'eyePositionCombinedWorld.y', 'eyePositionCombinedWorld.z','eyeDirectionCombinedWorld.y', 
 'eyeDirectionCombinedWorld.z', 'eyeDirectionCombinedLocal.x', 'eyeDirectionCombinedLocal.y', 'eyeDirectionCombinedLocal.z','hmdPosition.x', 'hmdPosition.y', 'hmdPosition.z', 'hmdDirectionForward.x', 'hmdDirectionForward.y', 'hmdDirectionForward.z', 
 'hmdRotation.x', 'hmdRotation.y', 'hmdRotation.z', 'hmdDirectionUp.x', 'hmdDirectionUp.y', 'hmdDirectionUp.z','playerBodyPosition.x', 'playerBodyPosition.y', 'playerBodyPosition.z', 'bodyTrackerPosition.x', 'bodyTrackerPosition.y', 
 'bodyTrackerPosition.z', 'bodyTrackerRotation.x', 'bodyTrackerRotation.y', 'bodyTrackerRotation.z', '0', 'hitColliderType', 'hitObjectColliderBoundsCenter', 'hitObjectColliderName', 'hitPointOnObject', 'ordinalOfHit']

In [17]:
One_participant.shape

(1982486, 110)

In [5]:
One_participant.tail()

,Unnamed: 0,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,hmdPosition.x,hmdPosition.y,...,bodyTrackerRotation.z,playerBodyPosition.x,playerBodyPosition.y,playerBodyPosition.z,0,hitColliderType,hitObjectColliderBoundsCenter,hitObjectColliderName,hitPointOnObject,ordinalOfHit
1982481,133286,1754,5,3,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 54.16472244262695,...",103.037399,0.900947,...,0.836601,103.085892,-0.670466,14.642349,NaN,UnityEngine.MeshCollider,"{'x': 38.798423767089844, 'y': 5.2340822219848...",TaskBuilding_30,"{'x': 40.99647903442383, 'y': 4.01120853424072...",2.0
1982482,133287,1754,5,3,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 39.85393524169922,...",102.993164,0.908930,...,0.941351,103.042618,-0.662711,14.691971,NaN,UnityEngine.MeshCollider,"{'x': 38.798423767089844, 'y': 5.2340822219848...",TaskBuilding_30,"{'x': 39.85393524169922, 'y': 3.67949295043945...",1.0
1982483,133288,1754,5,3,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 39.85393524169922,...",102.993164,0.908930,...,0.941351,103.042618,-0.662711,14.691971,NaN,UnityEngine.MeshCollider,"{'x': 33.59193420410156, 'y': 4.09291458129882...",Building_99,"{'x': 36.397789001464844, 'y': 3.8311033248901...",2.0
1982484,133289,1754,5,3,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 39.85393524169922,...",102.993164,0.908930,...,0.941351,103.042618,-0.662711,14.691971,NaN,UnityEngine.MeshCollider,"{'x': 38.798423767089844, 'y': 5.2340822219848...",TaskBuilding_30,"{'x': 39.85393524169922, 'y': 3.67949295043945...",1.0
1982485,133290,1754,5,3,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': 39.85393524169922,...",102.993164,0.908930,...,0.941351,103.042618,-0.662711,14.691971,NaN,UnityEngine.MeshCollider,"{'x': 33.59193420410156, 'y': 4.09291458129882...",Building_99,"{'x': 36.397789001464844, 'y': 3.8311033248901...",2.0


In [ ]:
out = pd.DataFrame([(n, k, v) for (n, d) in df.values for k, v in d.items()])
out.columns = ['Name', 'Subject', 'Marks']

In [8]:
One_participant = One_participant.sort_values(by=["SubjectID","Session","SessionSubsection"])

In [9]:
One_participant['rayCastHitsCombinedEyes'].explode().apply(pd.Series)

,0
73650,[{'hitPointOnObject': {'x': -62.10708236694336...
73651,[{'hitPointOnObject': {'x': -62.10708236694336...
73652,[{'hitPointOnObject': {'x': -62.05959320068359...
73653,[{'hitPointOnObject': {'x': -61.97558212280273...
73654,"[{'hitPointOnObject': {'x': -61.9382438659668,..."
...,...
73645,"[{'hitPointOnObject': {'x': 40.4099006652832, ..."
73646,"[{'hitPointOnObject': {'x': 54.16472244262695,..."
73647,"[{'hitPointOnObject': {'x': 54.16472244262695,..."
73648,"[{'hitPointOnObject': {'x': 39.85393524169922,..."


In [49]:
Try = pd.DataFrame(One_participant.head(100))

In [51]:
Try

,Unnamed: 0,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,hmdPosition.x,hmdPosition.y,...,hmdDirectionForward.y,hmdDirectionForward.z,hmdDirectionRight.x,hmdDirectionRight.y,bodyTrackerRotation.x,bodyTrackerRotation.y,bodyTrackerRotation.z,playerBodyPosition.x,playerBodyPosition.y,playerBodyPosition.z
0,0,1754,5,3,1.647345e+09,1.647345e+09,3,[{'hitPointOnObject': {'x': -314.2667541503906...,-312.048767,4.512682,...,-0.295788,-0.700996,-0.736998,0.015512,8.318716,214.530426,3.273872,-311.928711,2.948036,-7.825439
1,1,1754,5,3,1.647345e+09,1.647345e+09,3,[{'hitPointOnObject': {'x': -314.2667541503906...,-312.048767,4.512682,...,-0.295788,-0.700996,-0.736998,0.015512,8.318716,214.530426,3.273872,-311.928711,2.948036,-7.825439
2,2,1754,5,3,1.647345e+09,1.647345e+09,3,[{'hitPointOnObject': {'x': -314.4252014160156...,-312.048645,4.513359,...,-0.288868,-0.699886,-0.734154,0.015538,8.448868,214.284988,3.405638,-311.928711,2.948037,-7.825439
3,3,1754,5,3,1.647345e+09,1.647345e+09,3,[{'hitPointOnObject': {'x': -314.4252014160156...,-312.048645,4.513359,...,-0.288868,-0.699886,-0.734154,0.015538,8.448868,214.284988,3.405638,-311.928711,2.948037,-7.825439
4,4,1754,5,3,1.647345e+09,1.647345e+09,3,"[{'hitPointOnObject': {'x': -314.5205078125, '...",-312.048584,4.513680,...,-0.284591,-0.698855,-0.732036,0.015403,8.442637,214.147675,3.434256,-311.928711,2.948037,-7.825439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1754,5,3,1.647345e+09,1.647345e+09,0,[{'hitPointOnObject': {'x': -309.6007690429687...,-311.989868,4.538784,...,-0.061078,0.040227,0.038031,0.036700,0.415671,250.625916,4.803875,-311.928711,2.948037,-7.825439
96,96,1754,5,3,1.647345e+09,1.647345e+09,0,[{'hitPointOnObject': {'x': -309.5397033691406...,-311.988708,4.537722,...,-0.066552,0.061541,0.059444,0.033050,0.104511,251.379120,4.794288,-311.928711,2.948037,-7.825439
97,97,1754,5,3,1.647345e+09,1.647345e+09,0,[{'hitPointOnObject': {'x': -309.5397033691406...,-311.988708,4.537722,...,-0.066552,0.061541,0.059444,0.033050,0.104511,251.379120,4.794288,-311.928711,2.948037,-7.825439
98,98,1754,5,3,1.647345e+09,1.647345e+09,0,[{'hitPointOnObject': {'x': -309.5049438476562...,-311.988098,4.537337,...,-0.068633,0.074575,0.072504,0.032179,359.973022,251.875183,4.802982,-311.928711,2.948037,-7.825439


In [52]:
for index in range(len(Try)):
    pd.json_normalize(Try.at[index,'rayCastHitsCombinedEyes'][0])

AttributeError: 'str' object has no attribute 'values'

In [36]:
StartingPoints = pd.DataFrame(Try.explode('rayCastHitsCombinedEyes'))

In [45]:
one = exploded["rayCastHitsCombinedEyes"].values[0]
empty = pd.DataFrame()
empty = one

In [47]:
objects = Try["rayCastHitsCombinedEyes"].apply(json.loads)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [46]:
pd.json_normalize(one)

AttributeError: 'str' object has no attribute 'values'

In [15]:
Try['rayCastHitsCombinedEyes'][0]

"[{'hitPointOnObject': {'x': -314.2667541503906, 'y': 3.5156164169311523, 'z': -10.410945892333984}, 'hitObjectColliderName': 'Building_199', 'hitColliderType': 'UnityEngine.MeshCollider', 'hitObjectColliderBoundsCenter': {'x': -318.12432861328125, 'y': 7.002060890197754, 'z': -16.966527938842773}, 'ordinalOfHit': 1}, {'hitPointOnObject': {'x': -316.47393798828125, 'y': 2.5142099857330322, 'z': -12.716507911682129}, 'hitObjectColliderName': 'terrain_Q.001', 'hitColliderType': 'UnityEngine.MeshCollider', 'hitObjectColliderBoundsCenter': {'x': -347.9867858886719, 'y': 1.7923047542572021, 'z': 40.013404846191406}, 'ordinalOfHit': 2}]"

In [37]:
StartingPoints

,Unnamed: 0,SubjectID,Session,SessionSubsection,timeStampDataPointStart,timeStampDataPointEnd,combinedGazeValidityBitmask,rayCastHitsCombinedEyes,hmdPosition.x,hmdPosition.y,...,hmdDirectionForward.y,hmdDirectionForward.z,hmdDirectionRight.x,hmdDirectionRight.y,bodyTrackerRotation.x,bodyTrackerRotation.y,bodyTrackerRotation.z,playerBodyPosition.x,playerBodyPosition.y,playerBodyPosition.z
73650,73650,1754,1,1,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -62.10708236694336...,-59.117783,2.425328,...,-0.036818,0.987078,0.987091,0.031066,8.100518,355.064697,0.794874,-59.099869,0.858863,34.795731
73651,73651,1754,1,1,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -62.10708236694336...,-59.117783,2.425328,...,-0.036818,0.987078,0.987091,0.031066,8.100518,355.064697,0.794874,-59.099869,0.858863,34.795731
73652,73652,1754,1,1,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -62.05959320068359...,-59.117691,2.425254,...,-0.037479,0.987252,0.987287,0.031090,8.135050,355.152496,0.800912,-59.099869,0.858863,34.795731
73653,73653,1754,1,1,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -61.97558212280273...,-59.117599,2.425216,...,-0.038394,0.987469,0.987530,0.031259,8.127725,355.173431,0.789557,-59.099869,0.858863,34.795731
73654,73654,1754,1,1,1.646740e+09,1.646740e+09,3,"[{'hitPointOnObject': {'x': -61.9382438659668,...",-59.117447,2.425180,...,-0.038755,0.987628,0.987692,0.031540,8.043446,355.143066,0.795973,-59.099869,0.858863,34.795731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73745,73745,1754,1,1,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -68.32296752929688...,-59.117416,2.431289,...,-0.001439,0.992718,0.991848,0.041701,6.131139,355.322998,1.000091,-59.099869,0.858863,34.795731
73746,73746,1754,1,1,1.646740e+09,1.646740e+09,3,"[{'hitPointOnObject': {'x': -68.556396484375, ...",-59.117538,2.431200,...,-0.003633,0.992516,0.991618,0.042233,5.972204,355.387543,0.989731,-59.099869,0.858863,34.795731
73747,73747,1754,1,1,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -68.88323974609375...,-59.117691,2.431053,...,-0.004514,0.992237,0.991302,0.043072,5.824131,355.439453,0.981527,-59.099869,0.858863,34.795731
73748,73748,1754,1,1,1.646740e+09,1.646740e+09,3,[{'hitPointOnObject': {'x': -68.88323974609375...,-59.117691,2.431053,...,-0.004514,0.992237,0.991302,0.043072,5.824131,355.439453,0.981527,-59.099869,0.858863,34.795731


In [16]:
s = Try.pop('rayCastHitsCombinedEyes').explode()
df = Try.join(
    pd.DataFrame(s.tolist(), index=s.index))

In [38]:
TargetBuildingPosition = pd.DataFrame.from_records(Try.loc[:,'rayCastHitsCombinedEyes'])

KeyError: 0

In [ ]:
TargetBuildingPosition = pd.DataFrame.from_records(Try['rayCastHitsCombinedEyes'])
    dframe.insert(23, "TargetBuildingPosition_z",TargetBuildingPosition['z'], True)
    dframe.insert(23, "TargetBuildingPosition_y",TargetBuildingPosition['y'], True)
    dframe.insert(23, "TargetBuildingPosition_x",TargetBuildingPosition['x'], True)

In [29]:
df1 = pd.json_normalize(Try.at[Try.index,'rayCastHitsCombinedEyes'])            

TypeError: 'Int64Index([73650, 73651, 73652, 73653, 73654, 73655, 73656, 73657, 73658,
            73659, 73660, 73661, 73662, 73663, 73664, 73665, 73666, 73667,
            73668, 73669, 73670, 73671, 73672, 73673, 73674, 73675, 73676,
            73677, 73678, 73679, 73680, 73681, 73682, 73683, 73684, 73685,
            73686, 73687, 73688, 73689, 73690, 73691, 73692, 73693, 73694,
            73695, 73696, 73697, 73698, 73699, 73700, 73701, 73702, 73703,
            73704, 73705, 73706, 73707, 73708, 73709, 73710, 73711, 73712,
            73713, 73714, 73715, 73716, 73717, 73718, 73719, 73720, 73721,
            73722, 73723, 73724, 73725, 73726, 73727, 73728, 73729, 73730,
            73731, 73732, 73733, 73734, 73735, 73736, 73737, 73738, 73739,
            73740, 73741, 73742, 73743, 73744, 73745, 73746, 73747, 73748,
            73749],
           dtype='int64')' is an invalid key

In [28]:
Try.index

Int64Index([73650, 73651, 73652, 73653, 73654, 73655, 73656, 73657, 73658,
            73659, 73660, 73661, 73662, 73663, 73664, 73665, 73666, 73667,
            73668, 73669, 73670, 73671, 73672, 73673, 73674, 73675, 73676,
            73677, 73678, 73679, 73680, 73681, 73682, 73683, 73684, 73685,
            73686, 73687, 73688, 73689, 73690, 73691, 73692, 73693, 73694,
            73695, 73696, 73697, 73698, 73699, 73700, 73701, 73702, 73703,
            73704, 73705, 73706, 73707, 73708, 73709, 73710, 73711, 73712,
            73713, 73714, 73715, 73716, 73717, 73718, 73719, 73720, 73721,
            73722, 73723, 73724, 73725, 73726, 73727, 73728, 73729, 73730,
            73731, 73732, 73733, 73734, 73735, 73736, 73737, 73738, 73739,
            73740, 73741, 73742, 73743, 73744, 73745, 73746, 73747, 73748,
            73749],
           dtype='int64')

In [26]:
df = pd.json_normalize(Try['rayCastHitsCombinedEyes'])

AttributeError: 'str' object has no attribute 'values'